In [ ]:
import pandas as pd
import numpy as np 
import sklearn
import tensorflow_datasets as tfds
import tensorflow as tf 
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("/content/spam.csv", encoding = "ISO-8859-1")

In [ ]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
print(df.columns)
df = df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1)

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')


In [ ]:
df.head()
# "ham" messages are those that are not spam

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df.rename(columns = {"v1" : "type"}, inplace = True)
df.rename(columns = {"v2" : "text"}, inplace = True)

In [ ]:
df.head()

,type,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
(df["type"]).value_counts()

0    4825
1     747
Name: type, dtype: int64

In [ ]:
# We encode the values in column "type": 0 for non-spam messages, 1 for spam
df["type"] = df["type"].apply(lambda x: 0 if x=="ham" else 1)

In [ ]:
weights = 1/(df["type"]).value_counts()
weights = weights * len(df)/2
weights = {index : values for index , values in zip(weights.index,weights.values)}
weights

{0: 0.5774093264248704, 1: 3.72958500669344}

In [ ]:
df.head()

,type,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df['type'].nunique()

2

In [ ]:
df['text'].nunique()

5169

In [ ]:
# checking if there are missing values in the dataset
df.isnull().sum() 

type    0
text    0
dtype: int64

In [ ]:
!python -m spacy download en_core_web_lg -q

2022-12-06 16:03:21.526018: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
     |████████████████████████████████| 587.7 MB 16 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
import en_core_web_lg
nlp = en_core_web_lg.load()

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS

In [ ]:
df["text_clean"] = df["text"].apply(lambda x:''.join(ch for ch in x if ch.isalnum() or ch==" "))
df["text_clean"] = df["text_clean"].apply(lambda x: x.replace("  "," ").lower().strip())
df["text_clean"] = df["text_clean"].apply(lambda x: " ".join([token.lemma_ for token in nlp(x) if (token.lemma_ not in STOP_WORDS) & (token.text not in STOP_WORDS)]))

In [ ]:
df.head()

,type,text,text_clean
0,0,"Go until jurong point, crazy.. Available only ...",jurong point crazy available bugis n great wor...
1,0,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,0,U dun say so early hor... U c already then say...,u dun early hor u c
4,0,"Nah I don't think he goes to usf, he lives aro...",nah think usf live


In [ ]:
for i in range(0, 15):
  print(df["type"].loc[i], df["text"].loc[i])

0 Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
0 Ok lar... Joking wif u oni...
1 Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
0 U dun say so early hor... U c already then say...
0 Nah I don't think he goes to usf, he lives around here though
1 FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv
0 Even my brother is not like to speak with me. They treat me like aids patent.
0 As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
1 WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
1 Had your mobile 11 mont

In [ ]:
for i in range(0, 15):
  print(df["text_clean"].loc[i])

jurong point crazy available bugis n great world la e buffet cine amore wat
ok lar joke wif u oni
free entry 2 wkly comp win fa cup final tkts 21st 2005 text fa 87121 receive entry questionstd txt ratetcs apply 08452810075over18s
u dun early hor u c
nah think usf live
freemsg hey darle 3 week word d like fun tb ok xxx std chgs send å150 rcv
brother like speak treat like aids patent
request melle melle oru minnaminunginte nurungu vettam set callertune caller press 9 copy friend callertune
winner value network customer select receivea å900 prize reward claim 09061701461 claim code kl341 valid 12 hour
mobile 11 month u r entitle update late colour mobile camera free mobile update co free 08002986030
m home soon want talk stuff anymore tonight k ve cry today
chance win cash 100 20000 pound txt csh11 send 87575 cost 150pday 6days 16 tsandcs apply reply hl 4 info
urgent win 1 week free membership å100000 prize jackpot txt word claim 81010 tc wwwdbuknet lccltd pobox 4403ldnw1a7rw18
ve search 

In [ ]:
# we create a copy of the dataframe to look more closely 
spam_df = df[df["type"]=="spam"].copy()
spam_df = spam_df.reset_index(drop = True)

In [ ]:
"""for i in range(0, 15):
  print(spam_df["text"].loc[i])
  print()"""

KeyError: ignored

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(df["text_clean"])
df["text_encoded"] = tokenizer.texts_to_sequences(df["text_clean"])

In [ ]:
df.head()

,type,text,text_clean,text_encoded
0,0,"Go until jurong point, crazy.. Available only ...",jurong point crazy available bugis n great wor...,"[3602, 232, 446, 464, 943, 35, 51, 205, 944, 7..."
1,0,Ok lar... Joking wif u oni...,ok lar joke wif u oni,"[9, 193, 465, 287, 1, 1450]"
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,"[11, 296, 3, 534, 659, 33, 1451, 849, 423, 145..."
3,0,U dun say so early hor... U c already then say...,u dun early hor u c,"[1, 124, 150, 2363, 1, 83]"
4,0,"Nah I don't think he goes to usf, he lives aro...",nah think usf live,"[704, 22, 660, 130]"


In [ ]:
#df["text_padded_encoded"] = df["text_encoded"].apply(lambda x: tf.keras.preprocessing.sequence.pad_sequences(x, padding="post"))

In [ ]:
df.head()

,type,text,text_clean,text_encoded
0,0,"Go until jurong point, crazy.. Available only ...",jurong point crazy available bugis n great wor...,"[3602, 232, 446, 464, 943, 35, 51, 205, 944, 7..."
1,0,Ok lar... Joking wif u oni...,ok lar joke wif u oni,"[9, 193, 465, 287, 1, 1450]"
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,"[11, 296, 3, 534, 659, 33, 1451, 849, 423, 145..."
3,0,U dun say so early hor... U c already then say...,u dun early hor u c,"[1, 124, 150, 2363, 1, 83]"
4,0,"Nah I don't think he goes to usf, he lives aro...",nah think usf live,"[704, 22, 660, 130]"


In [ ]:
text_pad = tf.keras.preprocessing.sequence.pad_sequences(df["text_encoded"], padding="post")

In [ ]:
full_ds = tf.data.Dataset.from_tensor_slices((text_pad, df["type"]))

In [ ]:
df.shape

(5572, 4)

In [ ]:
TAKE_SIZE = int(0.7*df.shape[0])

train_data = full_ds.take(TAKE_SIZE).shuffle(TAKE_SIZE)
train_data = train_data.batch(64)

test_data = full_ds.skip(TAKE_SIZE)
test_data = test_data.batch(64)

In [ ]:
type(train_data.take(1))

tensorflow.python.data.ops.dataset_ops.TakeDataset

In [ ]:
for text, text_type in train_data.take(1):
  print(text, text_type)

tf.Tensor(
[[ 175   38    0 ...    0    0    0]
 [1030    3   10 ...    0    0    0]
 [ 116 6400 3383 ...    0    0    0]
 ...
 [ 107   34  105 ...    0    0    0]
 [   2  387 1320 ...    0    0    0]
 [ 220  158  177 ...    0    0    0]], shape=(64, 72), dtype=int32) tf.Tensor(
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0], shape=(64,), dtype=int64)


In [ ]:
type(text)

tensorflow.python.framework.ops.EagerTensor

In [ ]:
text.shape

TensorShape([64, 72])

In [ ]:
text.shape[1]

72

In [ ]:
text

In [ ]:
df.shape[1]

4

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM

vocab_size = len(tokenizer.word_index)
model_lstm = tf.keras.Sequential([
                  Embedding(vocab_size+1, 64, input_shape=[text.shape[1],],name="embedding"),
                  # text.shape[1]: 72
                  LSTM(units=64, return_sequences=True, name = "ltsm_1"), # maintains the sequential nature
                  LSTM(units=64, return_sequences=False, name = "lstm_2"), # returns the last output
                  Dense(16, activation='relu', name = "dense_1"),
                  Dense(8, activation='relu'),
                  Dense(1, activation="sigmoid", name="last")
                  ])

In [ ]:
model_lstm.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 72, 64)            525504    
                                                                 
 ltsm_1 (LSTM)               (None, 72, 64)            33024     
                                                                 
 lstm_2 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 16)                1040      
                                                                 
 dense_9 (Dense)             (None, 8)                 136       
                                                                 
 last (Dense)                (None, 1)                 9         
                                                                 
Total params: 592,737
Trainable params: 592,737
Non-tr

In [ ]:
optimizer= tf.keras.optimizers.Adam()

model_lstm.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

In [ ]:
model_lstm.fit(train_data,
              epochs=100, 
              validation_data=test_data,
               class_weight=weights)

Epoch 1/100
61/61 [==============================] - 12s 151ms/step - loss: 0.6910 - binary_accuracy: 0.8669 - val_loss: 0.6911 - val_binary_accuracy: 0.8636
Epoch 2/100
61/61 [==============================] - 8s 137ms/step - loss: 0.6910 - binary_accuracy: 0.8669 - val_loss: 0.6885 - val_binary_accuracy: 0.8636
Epoch 3/100
61/61 [==============================] - 8s 138ms/step - loss: 0.6912 - binary_accuracy: 0.8669 - val_loss: 0.6907 - val_binary_accuracy: 0.8636
Epoch 4/100
61/61 [==============================] - 8s 137ms/step - loss: 0.6910 - binary_accuracy: 0.8669 - val_loss: 0.6905 - val_binary_accuracy: 0.8636
Epoch 5/100
61/61 [==============================] - 8s 136ms/step - loss: 0.6910 - binary_accuracy: 0.8669 - val_loss: 0.6885 - val_binary_accuracy: 0.8636
Epoch 6/100
61/61 [==============================] - 8s 138ms/step - loss: 0.6910 - binary_accuracy: 0.8669 - val_loss: 0.6905 - val_binary_accuracy: 0.8636
Epoch 7/100
61/61 [==============================] - 8s 1

In [ ]:
model_lstm.save("model_lstm.h5")

In [ ]:
import json
json.dump(model_lstm.history.history, open("/content/LSTM_history.json", 'w'))

In [ ]:
LSTM_history = json.load(open("/content/LSTM_history.json", 'r'))
model_lstm = tf.keras.models.load_model("/content/model_lstm.h5")

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(y=LSTM_history["loss"],
                    mode='lines',
                    name='loss'))
fig.add_trace(go.Scatter(y=LSTM_history["val_loss"],
                    mode='lines',
                    name='val_loss'))
fig.show()
